**Imports**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import simplejson
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List, Dict, Tuple
from sklearn.linear_model import LinearRegression

**Code**

In [2]:
# import pandas as pd

# ======== بيانات وهمية ========
data = {
    "name": ["Ali", "Sara", "Mona", "Omar", "Khalid", "Lina", "Hassan", "Fatimah"],
    "income": [5000, 5200, 5100, 5300, 5250, 100000, 5150, 5050]  # لاحظ 100000 قيمة شاذة
}

df11 = pd.DataFrame(data)

print("=== البيانات الأصلية ===")
print(df11)

# ======== إزالة القيم الشاذة باستخدام Percentile Mask ========
# نحسب الحد الأعلى عند 99% (أي أي قيمة أعلى منه تعتبر شاذة)
upper_99 = df11["income"].quantile(0.99)

# نحدد الصفوف التي نحتفظ بها فقط
mask_keep = df11["income"] <= upper_99

# إنشاء نسخة جديدة بعد الحذف
df11_removed = df11[mask_keep].copy()

print("\n=== بعد إزالة القيم الشاذة ===")
print(df11_removed)

# عرض عدد الصفوف قبل وبعد الحذف
print("\nعدد الصفوف الأصلية:", len(df11))
print("عدد الصفوف بعد الإزالة:", len(df11_removed))

=== البيانات الأصلية ===
      name  income
0      Ali    5000
1     Sara    5200
2     Mona    5100
3     Omar    5300
4   Khalid    5250
5     Lina  100000
6   Hassan    5150
7  Fatimah    5050

=== بعد إزالة القيم الشاذة ===
      name  income
0      Ali    5000
1     Sara    5200
2     Mona    5100
3     Omar    5300
4   Khalid    5250
6   Hassan    5150
7  Fatimah    5050

عدد الصفوف الأصلية: 8
عدد الصفوف بعد الإزالة: 7


In [3]:
# import pandas as pd

# ======== بيانات وهمية ========
data = {
    "name": ["Ali", "Sara", "Mona", "Omar", "Khalid", "Lina", "Hassan", "Fatimah"],
    "income": [5000, 5200, 5100, 5300, 5250, 100000, 5150, 5050]  # 100000 قيمة شاذة
}

df = pd.DataFrame(data)

print("=== البيانات الأصلية ===")
print(df)

# ======== إزالة القيم الشاذة باستخدام IQR ========
Q1 = df["income"].quantile(0.25)
Q3 = df["income"].quantile(0.75)
IQR = Q3 - Q1

upper_iqr = Q3 + 1.5 * IQR
lower_iqr = Q1 - 1.5 * IQR

mask_iqr = (df["income"] >= lower_iqr) & (df["income"] <= upper_iqr)
df_iqr_removed = df[mask_iqr].copy()

print("\n=== بعد إزالة القيم الشاذة باستخدام IQR ===")
print(df_iqr_removed)

# ======== إزالة القيم الشاذة باستخدام Percentile 99 ========
upper_99 = df["income"].quantile(0.99)
mask_99 = df["income"] <= upper_99
df_percentile_removed = df[mask_99].copy()

print("\n=== بعد إزالة القيم الشاذة باستخدام Percentile 99 ===")
print(df_percentile_removed)

=== البيانات الأصلية ===
      name  income
0      Ali    5000
1     Sara    5200
2     Mona    5100
3     Omar    5300
4   Khalid    5250
5     Lina  100000
6   Hassan    5150
7  Fatimah    5050

=== بعد إزالة القيم الشاذة باستخدام IQR ===
      name  income
0      Ali    5000
1     Sara    5200
2     Mona    5100
3     Omar    5300
4   Khalid    5250
6   Hassan    5150
7  Fatimah    5050

=== بعد إزالة القيم الشاذة باستخدام Percentile 99 ===
      name  income
0      Ali    5000
1     Sara    5200
2     Mona    5100
3     Omar    5300
4   Khalid    5250
6   Hassan    5150
7  Fatimah    5050


In [4]:
# import pandas as pd

data = pd.Series([-3, 0, 5, 10, 20])

data_floored = data.clip(lower=0)
print(data_floored)

0     0
1     0
2     5
3    10
4    20
dtype: int64


In [5]:
# import pandas as pd

# ======== إنشاء بيانات وهمية ========
data = {
    "name": ["Ali", "Sara", "Mona", "Omar", "Khalid", "Lina", "Hassan", "Fatimah"],
    "income": [5000, 5200, 5100, 5300, 5250, 100000, 5150, 5050]  # لاحظ 100000 قيمة شاذة
}

df11 = pd.DataFrame(data)

# ======== دالة Capping عند النسبة المئوية ========
def cap_at_percentile(s: pd.Series, upper_q: float = 0.99) -> pd.Series:
    """
    Cap values in a numeric series at a given upper percentile.

    Parameters:
    s : pd.Series
        العمود الذي نريد قص القيم فيه.
    upper_q : float, default 0.99
        النسبة المئوية العليا لتحديد الحد الأعلى.

    Returns:
    pd.Series
        العمود بعد تطبيق القص.
    """
    upper = s.quantile(upper_q)  # حساب القيمة العليا عند النسبة المئوية
    return s.clip(upper=upper)   # قص القيم الأكبر من الحد

# ======== تطبيق الدالة على عمود income ========
df11["income_cap_99"] = cap_at_percentile(df11["income"], upper_q=0.99)

# ======== عرض النتائج ========
print("قبل القص:")
print(df11[["name", "income"]])
print("\nبعد القص عند النسبة 99%:")
print(df11[["name", "income_cap_99"]])

قبل القص:
      name  income
0      Ali    5000
1     Sara    5200
2     Mona    5100
3     Omar    5300
4   Khalid    5250
5     Lina  100000
6   Hassan    5150
7  Fatimah    5050

بعد القص عند النسبة 99%:
      name  income_cap_99
0      Ali           5000
1     Sara           5200
2     Mona           5100
3     Omar           5300
4   Khalid           5250
5     Lina          93371
6   Hassan           5150
7  Fatimah           5050


/var/folders/rj/0n0zqs0j3m30qjck5w9lh0980000gn/T/ipykernel_41013/4115749920.py:27: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return s.clip(upper=upper)   # قص القيم الأكبر من الحد
